In [32]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.preprocessing import PolynomialFeatures
from mlxtend.regressor import StackingCVRegressor

# Домашняя работа

1. Теперь решаем задачу регрессии - предскажем цены на недвижимость. Использовать датасет https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data (train.csv)
2. Данных немного, поэтому необходимо использовать 10-fold кросс-валидацию для оценки качества моделей
3. Построить случайный лес, вывести важность признаков
4. Обучить стекинг как минимум 3х моделей, использовать хотя бы 1 линейную модель и 1 нелинейную
5. Для валидации модели 2-го уровня использовать отдельный hold-out датасет, как на занятии
6. Показать, что использование ансамблей моделей действительно улучшает качество (стекинг vs другие модели сравнивать на hold-out)
7. В качестве решения:
    Jupyter notebook с кодом, комментариями и графиками

In [4]:
df = pd.read_csv('train_.csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
#Убираем пустые значения
df.LotFrontage = df.LotFrontage.fillna(0)
df.MasVnrArea = df.MasVnrArea.fillna(0)
df.GarageYrBlt = df.GarageYrBlt.fillna(0)
df = df.fillna('null')

In [6]:
#df.info()

In [7]:
x = df[[
'MSSubClass',
'MSZoning',
'LotFrontage',
'LotArea',
'Street',
'Alley',
'LotShape',
'LandContour',
'Utilities',
'LotConfig',
'LandSlope',
'Neighborhood',
'Condition1',
'Condition2',
'BldgType',
'HouseStyle',
'OverallQual',
'OverallCond',
'YearBuilt',
'YearRemodAdd',
'RoofStyle',
'RoofMatl',
'Exterior1st',
'Exterior2nd',
'MasVnrType',
'MasVnrArea',
'ExterQual',
'ExterCond',
'Foundation',
'BsmtQual',
'BsmtCond',
'BsmtExposure',
'BsmtFinType1',
'BsmtFinSF1',
'BsmtFinType2',
'BsmtFinSF2',
'BsmtUnfSF',
'TotalBsmtSF',
'Heating',
'HeatingQC',
'CentralAir',
'Electrical',
'1stFlrSF',
'2ndFlrSF',
'LowQualFinSF',
'GrLivArea',
'BsmtFullBath',
'BsmtHalfBath',
'FullBath',
'HalfBath',
'BedroomAbvGr',
'KitchenAbvGr',
'KitchenQual',
'TotRmsAbvGrd',
'Functional',
'Fireplaces',
'FireplaceQu',
'GarageType',
'GarageYrBlt',
'GarageFinish',
'GarageCars',
'GarageArea',
'GarageQual',
'GarageCond',
'PavedDrive',
'WoodDeckSF',
'OpenPorchSF',
'EnclosedPorch',
'3SsnPorch',
'ScreenPorch',
'PoolArea',
'PoolQC',
'Fence',
'MiscFeature',
'MiscVal',
'MoSold',
'YrSold',
'SaleType',
'SaleCondition']]

In [8]:
le = preprocessing.LabelEncoder()

In [9]:
colm = x.columns

In [10]:
#фит трансфорт всего х - переводим стринги в инты
x = x.apply(le.fit_transform) 
#x = x.iloc[:, 0].values
#x = x.reshape(-1,1)

In [11]:
y = df.SalePrice

In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.3)

In [13]:
rgs = RandomForestRegressor(n_estimators=100, random_state=0
                            #,oob_score=True
                            #,bootstrap=True
                           )

In [14]:
#учим лес
rgs.fit(X_train,Y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [15]:
#важность признаков
feature_importances = pd.DataFrame(rgs.feature_importances_, index=colm, columns=['importances']).sort_values('importances', ascending=False)
feature_importances

,importances
OverallQual,5.317186e-01
GrLivArea,1.238537e-01
TotalBsmtSF,4.462419e-02
2ndFlrSF,3.388393e-02
LotArea,2.940753e-02
...,...
Heating,2.718888e-05
PoolQC,2.656706e-05
MiscFeature,2.585882e-05
Street,1.190369e-06


In [16]:
y_pred_test = rgs.predict(X_test)

In [17]:
rgs.score(X_train,Y_train)

0.9764578370738812

In [18]:
rgs.score(X_test,Y_test)

0.8944405113772598

In [19]:
#metrics.r2_score(Y_test,y_pred_test)

In [20]:
num_folds = 10

In [21]:
kfold_ = KFold(n_splits=num_folds)

In [22]:
cros_val_rgs = cross_val_score(rgs,X_train,Y_train,cv=kfold_)

In [23]:
#оценка качества модели на фолдах = 10. Проверка на переобучение
cros_val_rgs.mean()

0.8523107701235016

In [24]:
#Линейная регрессия

In [25]:
lgs = LinearRegression()

In [26]:
lgs.fit(X_train,Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [27]:
y_pred_test = lgs.predict(X_test)

In [28]:
lgs.score(X_train,Y_train)

0.8556977140271319

In [29]:
lgs.score(X_test,Y_test)

0.8333032948382862

In [30]:
cros_val_lgs = cross_val_score(lgs,X_train,Y_train,cv=kfold_)

In [31]:
cros_val_lgs.mean()

0.8012852847614255

In [41]:
#не строим свой велосипед, а используем готовое решение для стекинга
svr = SVR(kernel='linear') #модель 1
lr = LinearRegression() #модель 2
rf = RandomForestRegressor(n_estimators=100, 
                           random_state=0) #модель 1

#стекинг
stack = StackingCVRegressor(regressors=(svr, lr, rf), #обучаем наши 3 модели
                            meta_regressor=lr, #передаем их метарегрессору
                            cv=10, #фолды
                            random_state=0)

print('10-fold cross validation scores:\n')

for clf, label in zip([svr, lr, rf, stack], ['SVM', 'LinearRegression', 
                                                'Random Forest', 
                                                'StackingCVRegressor']):
    scores = cross_val_score(clf, x, y, cv=10) #фолды на кросс валидацию
    print("R^2 Score: %0.5f (+/- %0.5f) [%s]" % (
        scores.mean(), scores.std(), label))

10-fold cross validation scores:

R^2 Score: 0.74246 (+/- 0.06439) [SVM]
R^2 Score: 0.83071 (+/- 0.06299) [LinearRegression]
R^2 Score: 0.86512 (+/- 0.04246) [Random Forest]
R^2 Score: 0.86937 (+/- 0.04925) [StackingCVRegressor]


In [43]:
#ВИдим что стекинг дает самый лучший результат, но его отличие от рендоп фореста на самом деле очень не большое
#Как понимаю тут вопрос больше к данным и их предобработки + по хорошему это все нужно учить\тестить на отдельных выборках

In [39]:
for clf, label in zip([svr, lr, rf, stack], ['SVM', 'LinearRegression', 
                                                'Random Forest', 
                                                'StackingCVRegressor']):
    scores = cross_val_score(clf, x, y, cv=10, scoring='neg_mean_squared_error')
    print("Neg. MSE Score: %0.2f (+/- %0.2f) [%s]" % (
        scores.mean(), scores.std(), label))

Neg. MSE Score: -1674230148.03 (+/- 767141729.62) [SVM]
Neg. MSE Score: -1112992078.31 (+/- 629302574.24) [LinearRegression]
Neg. MSE Score: -864872644.91 (+/- 398832985.78) [Random Forest]
Neg. MSE Score: -847598107.71 (+/- 457998798.24) [StackingCVRegressor]


In [40]:
#http://rasbt.github.io/mlxtend/user_guide/regressor/StackingCVRegressor/